<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH19_05_%ED%95%9C%EA%B5%AD%EC%96%B4_%ED%82%A4%EB%B2%84%ED%8A%B8(Korean_KeyBERT)%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=48a2d0c5f824e2f1216077c116ab494b0f555ce56b77d2a2b246b99571f0a9f4
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 1. 기본 KeyBERT


In [13]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
doc = """
         드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
         서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
         드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
         영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
         드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
         지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
         이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
         소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
         유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
         "향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
      """


In [4]:
okt = Okt()
tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print(tokenized_doc[:10])
tokenized_nouns

[('\n         ', 'Foreign'), ('드론', 'Noun'), ('활용', 'Noun'), ('범위', 'Noun'), ('도', 'Josa'), ('점차', 'Noun'), ('확대', 'Noun'), ('되고', 'Verb'), ('있다', 'Adjective'), ('.', 'Punctuation')]


'드론 활용 범위 점차 확대 최근 미세먼지 관리 드론 활용 서울시 미세먼지 계절 관리제 기간 지난달 개 월간 드론 측정 장치 달 미세먼지 집중 관리 실시 드론 산업 단지 사업 밀집 지역 미세먼지 배출 수치 점검 현장 모습 영상 영상 통해 미세먼지 방지 시설 제대로 가동 업체 무허가 시설 대한 단속 한층 전망 드론 활용 가장 적극 소방청 복합 재난 대응 차원 드론 관련 전문 인력 보강 어가 지난해 말 기준 소방청 보유 드론 총 드론 조종 자격증 소방대 경우 명 이 중 실기 평가 지도 자격증 드론 전문가 명도 배치 소방청 관계자 소방 드론 재난 현장 영상 정보 수집 산악 수난 사고 시 인명 수색 구조 활동 유독가스 폭발사고 시 대원 안전 확보 등 활용 며 향후 화재 진압 인명구조 등 드론 활용 위해 연구개발 고 말'

n_gram_range를 이용한 단어 추출

In [5]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수:', len(candidates))
print('상위 다섯개만 출력;', candidates[:5])

trigram 개수: 222
상위 다섯개만 출력; ['가동 업체' '가동 업체 무허가' '가장 적극' '가장 적극 소방청' '경우 실기']


문서와 키워드를 SBERT를 이용해 수치화함. 한국어를 포함하고 있는 다국어 SBERT를 이용함

In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [7]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

문서와 가장 유사한 키워드 상위 5개 추출함. 

In [8]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['드론 산업', '드론 드론 조종', '실시 드론 산업', '관리 드론 활용', '미세먼지 관리 드론']

마찬가지로 다양한 키워드를 얻기 위해 두가지 알고리즘 사용
* Max Sum Similarity
* Maximal Marginal Relevance

### 2. Max Sum Similarity
------------


In [9]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
  distances = cosine_similarity(doc_embedding, candidate_embeddings)

  # 키워드간 유사도
  candidate_distances = cosine_similarity(candidate_embeddings, candidate_embeddings)

  # 상위 nr_candidates개 후보 키워드 도출
  words_idx = list(distances.argsort()[0][-nr_candidates:])
  words_vals = [candidates[index] for index in words_idx]
  candidate_distances = candidate_distances[np.ix_(words_idx, words_idx)]

  # 각 키워드들 중 가장 덜 유사한 키워드들간 조합 계산
  min_sim = np.inf
  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([candidate_distances[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim
  return [words_vals[idx] for idx in candidate]

In [10]:
# 상위 10개 키워드를 선택한 후 그 중 유사성이 낮은 5개 선정
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['드론 산업 단지', '전망 드론 활용', '드론 산업', '관리 드론 활용', '미세먼지 관리 드론']

In [11]:
# 상위 30개 키워드를 선택한 후 그 중 유사성이 낮은 5개 선정
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['소방 드론 재난', '자격증 드론 전문가', '월간 드론 측정', '전망 드론 활용', '미세먼지 관리 드론']

### 3. Maximal Marginal Relvance(MMR 방법)
----------------

In [31]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
  word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

  # 키워드간 유사도
  word_similarity = cosine_similarity(candidate_embeddings, candidate_embeddings)

  # 문서와 가장 높은 유사도를 가진 키워드 인덱스 추출
  keywords_idx = [np.argmax(word_doc_similarity)]

  # 가장 높은 유사도 키워드를 제외한 키워드 인덱스
  candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
  
  # top_n-1번 만큼 반복
  for _ in range(top_n - 1):
    candidate_similarity = word_doc_similarity[candidates_idx, :]
    target_similarity = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

    # mmr 계산
    mmr = (1-diversity) * candidate_similarity - diversity * target_similarity.reshape(-1, 1)
    mmr_idx = candidates_idx[np.argmax(mmr)]

    # kewywords & candidate 업데이트
    keywords_idx.append(mmr_idx)
    candidates_idx.remove(mmr_idx)
  return [words[idx] for idx in keywords_idx]

In [32]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['미세먼지 관리 드론', '실시 드론 산업', '관리 드론 활용', '월간 드론 측정', '전망 드론 활용']

In [33]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['미세먼지 관리 드론', '사업 밀집', '재난 현장 영상', '산악 수난', '수치 점검']